<table class="ee-notebook-buttons" align="center">
    <td><a target="_blank"  href="https://colab.research.google.com/github/ytarazona/lomas-pnud/blob/main/notebooks/03%20Cuantificaci%C3%B3n%20de%20cobertura%20de%20vegetaci%C3%B3n%20de%20Lomas.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## 1. Importación de las bibliotecas necesarias

In [ ]:
# Instalar geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [1]:
import ee
import numpy as np
import pandas as pd
import geemap as emap
Map = emap.Map()
Map.add_basemap('ROADMAP')

## 2. Cuantificación de cobertura de vegetación de Lomas

### 2.1 Cuantificación de cobertura para la loma "Lurín"

In [2]:
# Establecemos el poligono de lomas mapeadas
lurin = ee.FeatureCollection('users/geoyons/tugee/Lomas_Modis')\
                  .filterMetadata('Nombre','equals','Lurin')

In [3]:
## VISUALIZACION DE LAS HECTAREAS DE UNA LOMA ESPECIFICA - LOMA PACHACAMAC

# Definimos la coleccion
collection = 'MODIS/006/MOD13Q1'

# Definimos las bandas
bands = ['NDVI']
# Para umbrales menores a 0.15 de NDVI (no vegetacion)
value = 2

# Funcion de filtro de pixeles contaminados usando la banda de fiabilidad
def maskSQA(image):
    mask = image.select('SummaryQA').eq(0).Or(image.select('SummaryQA').eq(1))
    imageMask = image.updateMask(mask)
    return imageMask
    
# Filtrado de la coleccion MODIS
rast = ee.ImageCollection(collection)\
        .filterDate('2002-07-01', '2002-10-30')\
        .filterBounds(lurin)\
        .map(maskSQA)\
        .reduce(ee.Reducer.median()).select([0], bands)\
        .reproject(crs = 'EPSG:4326', scale = 250)

haLomas = rast.divide(10000).gte(0.15)
mask = haLomas.lt(0.15)
haLomas = mask.multiply(value).add(haLomas.multiply(mask.Not()))\
              .unmask(3).clip(lurin) # 3 para areas de nubes o residuales

# Definimos los parametros de visualizacion
visParams = {
  'bands': ['NDVI'],
  'min': 1,
  'max': 3,
  'palette': ['#13de13','#4d4def','#dc0f0f'] # cod1 = veget, cod2=NoVeget, cod3=cobNubes
}

# Visualizamos
Map.centerObject(lurin, 13)
Map.addLayer(lurin, {}, 'Variante de Ancon', True)
Map.addLayer(haLomas, visParams, 'Areas de clases Variante de Ancon')
Map

Map(center=[-12.240160663076535, -76.76200436621019], controls=(WidgetControl(options=['position'], widget=HBo…

### 2.2 Descarga de las áreas de todas las lomas

In [ ]:
## DESCARGA DE LAS AREAS DE CADA LOMA ENTRE EL PERIODO 2000 A 2020

# Definimos las entidades vectoriales
lomas = ee.FeatureCollection('users/geoyons/tugee/Lomas_Modis')
#listLomas = lomas.toList(lomas.size())
#listLomas.length().getInfo()

# Leer la tabla de nombres de lomas con PANDAS
data = pd.read_csv("F:/ConsultoriaPNUD/Data/CSV/Lomas_Modis.csv", delimiter = ";")
# Para ver los primeros 3
names = data['Nombre'].tolist() # convertir toda el df => df.values.tolist()

# Definimos la coleccion
collection = 'MODIS/006/MOD13Q1'

# Definimos las bandas
bands = ['NDVI']
# Para umbrales menores a 0.15 de NDVI
value = 2

ini_date = ['2000-07-01','2001-07-01','2002-07-01','2003-07-01','2004-07-01','2005-07-01','2006-07-01','2007-07-01',
           '2008-07-01','2009-07-01','2010-07-01','2011-07-01','2012-07-01','2013-07-01','2014-07-01','2015-07-01',
           '2016-07-01','2017-07-01','2018-07-01','2019-07-01','2020-07-01']

end_date = ['2000-10-30','2001-10-30','2002-10-30','2003-10-30','2004-10-30','2005-10-30','2006-10-30','2007-10-30',
           '2008-10-30','2009-10-30','2010-10-30','2011-10-30','2012-10-30','2013-10-30','2014-10-30','2015-10-30',
           '2016-10-30','2017-10-30','2018-10-30','2019-10-30','2020-10-30']

# Funcion de filtro de pixeles contaminados usando la banda de fiabilidad
def maskSQA(image):
    mask = image.select('SummaryQA').eq(0).Or(image.select('SummaryQA').eq(1))
    imageMask = image.updateMask(mask)
    return imageMask

for j in np.arange(0, len(names), 1):
    
    clasLomas = ee.List([])
    
    #name = ee.Feature(listLomas.get(j)).getInfo()["properties"]['NOMBRE'] # NOMBRE es el campo
    
    lms = ee.FeatureCollection('users/geoyons/tugee/Lomas_Modis')\
            .filterMetadata('Nombre','equals', names[j])
    
    for i in np.arange(0, len(ini_date), 1):
        # Filtrado de la coleccion MODIS
        rast = ee.ImageCollection(collection)\
             .filterDate(ini_date[i], end_date[i])\
             .filterBounds(lms)\
             .map(maskSQA)\
             .reduce(ee.Reducer.max()).select([0], bands)
        
        haLomas = rast.divide(10000).gte(0.15)
        mask = haLomas.lt(0.15)
        haLomas = mask.multiply(value).add(haLomas.multiply(mask.Not()))\
              .unmask(3).clip(lms) # 3 para areas de nubes o residuales
        
        clasLomas = clasLomas.add(haLomas)
    
    # Coleccion de imagenes
    clasLomas = ee.ImageCollection.fromImages(clasLomas)
    # Coleccion a stack
    clasLomas = clasLomas.toBands()
    
    # Descargar directamente a Google Drive
    task = ee.batch.Export.image(clasLomas.divide(1), 
                              'LOMA_' + names[j] + '_' + '2000-2020' + '_CLASS', 
                              {'region': lms.geometry(),
                               'scale': 250,
                               'maxPixels': 1e13,
                               'folder': 'MOD13Q1_NDVI',
                               'fileFormat': 'GeoTIFF',
                               'crs': 'EPSG:4326'})
    print('Descargando la imagen:' + 'LOMA_' + names[j] + '_' + '2000-2020' + '_CLASS' + '.tif')
    task.start()